In [ ]:
import os
import time
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchsummary import summary

# Import your modules
from data.data_generator import main as generate_data, load_and_stack_datasets
from data.data_loader import load_data, MyDataset
from models.model import CustomNN
from utils.training import train

In [33]:
# Configuration dictionary
config = {
    'input_size': 1851,
    'output_size': 1800,
    'hidden_layers': 4,
    'neurons_per_layer': [3072],  # Base neuron count; will be repeated hidden_layers times.
    'dropout_prob': 0.0,
    'use_batch_norm': False,
    'activation': 'leaky_relu',
    'learning_rate': 1e-4,
    'epochs': 51,
    'batch_size': 1024,
    'model_save_prefix': './'
}

print("Configuration loaded:")
print(config)

Configuration loaded:
{'input_size': 1851, 'output_size': 1800, 'hidden_layers': 4, 'neurons_per_layer': [3072], 'dropout_prob': 0.0, 'use_batch_norm': False, 'activation': 'leaky_relu', 'learning_rate': 0.0001, 'epochs': 51, 'batch_size': 1024, 'model_save_prefix': './'}


In [5]:
%load_ext autoreload
%autoreload 2

from data.data_generator import main as generate_data, load_and_stack_datasets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# List of data files (updated names)
data_files = [
    'data/xtrain_nl2.pt',
    'data/ytrain_nl2.pt',
    'data/x_valid_nl2.pt',
    'data/y_valid_nl2.pt',
    'data/norm_data_nl2.nc'
]

# Check if data files exist; if not, generate them.
if not all(os.path.exists(f) for f in data_files):
    print("Generating data...")
    generate_data()  # This calls your data_generator.py main() function
    print("Data generation complete.")
else:
    print("Data already generated. Skipping generation.")


Generating data...
<xarray.DataArray 'CG' (nk: 50, time_station: 15310)>
array([[ 2.6327693 ,  2.6473465 ,  2.6327693 , ...,  2.6327693 ,
         2.6327693 ,  2.6327693 ],
       [ 2.3880813 ,  2.3929522 ,  2.3880813 , ...,  2.3880813 ,
         2.3880813 ,  2.3880813 ],
       [ 2.159401  ,  2.1607904 ,  2.159401  , ...,  2.159401  ,
         2.159401  ,  2.159401  ],
       ...,
       [-0.95191115, -0.95191115, -0.95191115, ..., -0.95191115,
        -0.95191115, -0.95191115],
       [-0.96208704, -0.96208704, -0.96208704, ..., -0.96208704,
        -0.96208704, -0.96208704],
       [-0.9715973 , -0.9715973 , -0.9715973 , ..., -0.9715973 ,
        -0.9715973 , -0.9715973 ]], dtype=float32)
Coordinates:
  * time_station  (time_station) object MultiIndex
  * time          (time_station) datetime64[ns] 2010-01-03 ... 2010-01-02T12:...
  * station       (time_station) int64 21601 32946 31468 ... 29293 36567 38599
Dimensions without coordinates: nk
<xarray.DataArray 'VA' (nspec: 1800, tim

In [22]:
# Load the data using your data loader module
x, y, x_valid, y_valid, norm_data = load_data()

# Create dataset objects
train_dataset = MyDataset(x, y)
valid_dataset = MyDataset(x_valid, y_valid)

# Create DataLoaders using the batch size from config
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=False, drop_last=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=False, drop_last=True)

print("Data loaded and DataLoaders created.")


Data loaded and DataLoaders created.


In [23]:
# Repeat base neuron count by number of hidden layers
neurons_list = config['neurons_per_layer'] * config['hidden_layers']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Build the model using your CustomNN class
model = CustomNN(
    config['input_size'],
    config['output_size'],
    config['hidden_layers'],
    neurons_list,
    config['dropout_prob'],
    config['use_batch_norm'],
    config['activation']
).to(device)

# Display model summary
summary(model, (config['batch_size'], config['input_size']))
print("Model built on device:", device)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1           [-1, 1024, 3072]       5,689,344
         LeakyReLU-2           [-1, 1024, 3072]               0
            Linear-3           [-1, 1024, 3072]       9,440,256
         LeakyReLU-4           [-1, 1024, 3072]               0
            Linear-5           [-1, 1024, 3072]       9,440,256
         LeakyReLU-6           [-1, 1024, 3072]               0
            Linear-7           [-1, 1024, 3072]       9,440,256
         LeakyReLU-8           [-1, 1024, 3072]               0
            Linear-9           [-1, 1024, 3072]       9,440,256
        LeakyReLU-10           [-1, 1024, 3072]               0
           Linear-11           [-1, 1024, 1800]       5,531,400
Total params: 48,981,768
Trainable params: 48,981,768
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 7.23
Fo

In [24]:
# Create a normalization tensor from norm_data
fac = 1e6
a1 = (fac * norm_data['VNL_std'] + 0).transpose().values
norm_tensor = torch.FloatTensor(a1).to(device)

def criterion(output, target):
    return torch.mean(torch.abs(norm_tensor * (target - output)))

print("Custom loss function defined.")


Custom loss function defined.


In [32]:
# Create dynamic model save name using config parameters:
model_save_name = f"{config['model_save_prefix']}L{config['hidden_layers']}_{neurons_list[0]}_"
model_save_name

'./L4_3072_'

In [34]:
# Set up the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], amsgrad=True)

# Train the model and record training results
start_time = time.time()
training_results = train(
    model, criterion, train_dataloader, valid_dataloader,
    optimizer, config['epochs'], model_save_name, device, save_epochs=[5]
)
elapsed = time.time() - start_time
print("Training completed in {:.2f} seconds".format(elapsed))


Epochs:  1 / 51
Model saved at epoch 5
Epochs:  10 / 51
Epochs:  20 / 51
Epochs:  30 / 51
Epochs:  40 / 51
Epochs:  50 / 51
Training completed in 3.07 seconds


In [27]:
model_save_name = f"{config['model_save_prefix']}L{config['hidden_layers']}_{neurons_list[0]}_{config['epochs']}epoch"
 
# Save training logs
np.savetxt(model_save_name + '_tr.txt', np.array(training_results['train_loss']))
np.savetxt(model_save_name + '_vd.txt', np.array(training_results['valid_loss']))

In [28]:
config['epochs']

301